# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,23.50,76,100,6.14,PN,1704694236
1,1,grytviken,-54.2811,-36.5092,3.88,88,50,1.53,GS,1704694238
2,2,guarapari,-20.6667,-40.4975,24.94,88,47,2.27,BR,1704694239
3,3,ust-nera,64.5667,143.2000,-43.84,98,81,1.55,RU,1704694239
4,4,puri,19.8000,85.8500,23.29,45,13,4.07,IN,1704694240


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_hvplot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles = 'OSM',
    frame_width = 900,
    frame_height = 600,
    size = 'Humidity',
    scale = 1,
    color = 'City'
)

# Display the map
city_data_hvplot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
vacation_df = city_data_df.loc[(city_data_df['Max Temp']>22) & (city_data_df['Humidity']<80) & (city_data_df['Cloudiness']<80)]

# Drop any rows with null values
vacation_df = vacation_df.dropna(how='any')

# Display sample data
vacation_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,puri,19.8000,85.8500,23.29,45,13,4.07,IN,1704694240
5,5,uturoa,-16.7333,-151.4333,27.00,79,64,3.70,PF,1704694241
14,14,maun,-19.9833,23.4167,27.98,54,0,2.57,BW,1704694247
19,19,abepura,-2.6333,140.5833,31.45,62,20,0.51,ID,1704694250
22,22,west island,-12.1568,96.8225,28.99,79,75,6.17,CC,1704694252
...,...,...,...,...,...,...,...,...,...,...
536,536,inhambane,-23.8650,35.3833,27.80,62,18,1.18,MZ,1704694641
543,543,abu jibeha,11.4562,31.2285,23.04,25,22,2.10,SD,1704694646
545,545,ta`u,-14.2336,-169.5144,28.67,73,71,1.51,AS,1704694649
555,555,sanawad,22.1833,76.0667,27.50,40,20,3.36,IN,1704694656


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_df.copy()
dropped_col = ['City_ID', 'Max Temp', 'Cloudiness', 'Wind Speed', 'Date']
hotel_df = hotel_df.drop(dropped_col, axis=1)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=  " "

# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
4,puri,19.8000,85.8500,45,IN,
5,uturoa,-16.7333,-151.4333,79,PF,
14,maun,-19.9833,23.4167,54,BW,
19,abepura,-2.6333,140.5833,62,ID,
22,west island,-12.1568,96.8225,79,CC,
...,...,...,...,...,...,...
536,inhambane,-23.8650,35.3833,62,MZ,
543,abu jibeha,11.4562,31.2285,25,SD,
545,ta`u,-14.2336,-169.5144,73,AS,
555,sanawad,22.1833,76.0667,40,IN,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
limit = 5
categories = 'accommodation.hotel'
params = {
    'categories': categories,
    'limit':limit,
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, 'Lat']
    lon = hotel_df.loc[index,'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
puri - nearest hotel: Divine Beach Homestay
uturoa - nearest hotel: Hawaiki Nui hotel
maun - nearest hotel: Center Lodge Conference Center
abepura - nearest hotel: Yoyo
west island - nearest hotel: Cocos Village Bungalows
taunggyi - nearest hotel: Golden Island Cottages
brisas de zicatela - nearest hotel: Casa de Olas
toliara - nearest hotel: Ambary
sikasso - nearest hotel: No hotel found
hithadhoo - nearest hotel: Pebbles Inn
malvan - nearest hotel: Hotel Silver Sand
carnarvon - nearest hotel: No hotel found
port pirie - nearest hotel: Ellen Hotel
karratha - nearest hotel: Karratha International Hotel
pasni - nearest hotel: Juddi Hotel
tura - nearest hotel: No hotel found
georgetown - nearest hotel: Page 63 hostel
mandasa - nearest hotel: No hotel found
jinghong - nearest hotel: โรงแรมสืเบา
port mathurin - nearest hotel: Escale Vacances
saint-joseph - nearest hotel: PALM Hotel & Spa
bubaque - nearest hotel: Chez Julio
rongelap - nearest hotel: No hotel found
san 

,City,Lat,Lng,Humidity,Country,Hotel Name
4,puri,19.8000,85.8500,45,IN,Divine Beach Homestay
5,uturoa,-16.7333,-151.4333,79,PF,Hawaiki Nui hotel
14,maun,-19.9833,23.4167,54,BW,Center Lodge Conference Center
19,abepura,-2.6333,140.5833,62,ID,Yoyo
22,west island,-12.1568,96.8225,79,CC,Cocos Village Bungalows
...,...,...,...,...,...,...
536,inhambane,-23.8650,35.3833,62,MZ,Hotel de Inhambane
543,abu jibeha,11.4562,31.2285,25,SD,No hotel found
545,ta`u,-14.2336,-169.5144,73,AS,No hotel found
555,sanawad,22.1833,76.0667,40,IN,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot

hotel_hvplot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles = 'OSM',
    frame_width = 900,
    frame_height = 500,
    size = 'Humidity',
    scale = 1,
    color = 'City',
    hover_cols = ['Hotel Name', 'Country']
)
# Display the map
hotel_hvplot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)